In [1]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='root', password='1030', db='zero_pay', charset='utf8')
cur = conn.cursor()

In [4]:
import pandas as pd

In [20]:
restau = pd.read_sql("SELECT * FROM restaurant", conn)
restau.head()
restau_names = restau['place_name']
restau_names


C:\Users\user\AppData\Local\Temp\ipykernel_31000\1994455848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  restau = pd.read_sql("SELECT * FROM restaurant", conn)


0         숯불돼지갈비마포집
1             황푸차이나
2            명일원조닭발
3             전주감자탕
4      양도령숯불민물장어 본점
           ...     
295    아우네쭈꾸미꼼장어 본점
296           대가흑염소
297             양자강
298          형제기사식당
299          학생회관분식
Name: place_name, Length: 300, dtype: object

In [29]:
import requests
import csv

restau = pd.read_sql("SELECT * FROM restaurant", conn)
restau.head()
restau_names = restau['place_name'].tolist()

# 카카오 API 설정
KAKAO_API_KEY = '65cfa5251145011a54255c8c67074e68'  # 카카오 REST API 키

# 카카오 로컬 API를 사용한 장소 검색 함수
def search_places(query, page=1, size=15):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": query,  # 검색할 키워드
        "page": page,    # 페이지 번호
        "size": size     # 한 페이지에 보여질 문서 수 (최대 15)
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# 모든 검색 결과를 수집하는 함수
def get_all_places(query, max_pages=5):
    all_places = []
    for page in range(1, max_pages + 1):
        print(f"Fetching page {page}...")
        data = search_places(query, page=page)
        if not data or 'documents' not in data:
            break
        places = data['documents']
        if not places:
            print("No more results.")
            break
        all_places.extend(places)
    return all_places




# 수집된 데이터를 CSV 파일로 저장하는 함수
def save_to_csv(places, filename='restaurant.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['place_name', 'address_name', 'road_address_name', 'x', 'y']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for place in places:
            writer.writerow({
                'place_name': place.get('place_name'),
                'address_name': place.get('address_name'),
                'road_address_name': place.get('road_address_name'),
                'x': place.get('x'),  # 경도
                'y': place.get('y')   # 위도
            })
    print(f"Saved {len(places)} records to {filename}")

    


    
    
# 메인 실행 부분
if __name__ == "__main__":
    query = "강동구 제로페이 식당"  # 검색 키워드 설정
    max_pages = 5            # 크롤링할 최대 페이지 수 설정
    places = get_all_places(query, max_pages=max_pages)
    print(f"Total places found: {len(places)}")
    save_to_csv(places)
    

C:\Users\user\AppData\Local\Temp\ipykernel_31000\913996811.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  restau = pd.read_sql("SELECT * FROM restaurant", conn)


Saved 61 records to test.csv
